In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from scipy.sparse import hstack
from sklearn.metrics import roc_auc_score

In [2]:
%%time
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

target = pd.read_csv('../input/train.csv.zip').fillna(' ')[class_names].values
target

CPU times: user 802 ms, sys: 59.9 ms, total: 861 ms
Wall time: 860 ms


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [3]:
train_features = np.load('../input/train_embs_large_400.npy')
test_features = np.load('../input/test_embs_large_400.npy')

In [5]:
%%time
train_oof = np.zeros(target.shape)
kf = KFold(n_splits=3, random_state=137, shuffle=True)
for ii in range(6):
    print("Fitting target", ii+1)
    for jj, (train_index, val_index) in enumerate(kf.split(train_features)):
        print("Fitting fold", jj+1)
        train_x = train_features[train_index]
        val_x = train_features[val_index]
        train_target = target[train_index,ii]
        classifier = XGBClassifier(tree_method='hist')
        classifier.fit(train_x, train_target)
        train_oof[val_index, ii] = classifier.predict_proba(val_x)[:,1]
    print(roc_auc_score(target[:,ii], train_oof[:,ii]))

Fitting target 1
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9589959652165291
Fitting target 2
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9795291987529393
Fitting target 3
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9682684632688761
Fitting target 4
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9637029624024283
Fitting target 5
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.964538749598732
Fitting target 6
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9535626675933024
CPU times: user 9h 34min 24s, sys: 45.5 s, total: 9h 35min 10s
Wall time: 38min 39s


In [6]:
%%time
train_oof = np.zeros(target.shape)
kf = KFold(n_splits=3, random_state=137, shuffle=True)
for ii in range(6):
    print("Fitting target", ii+1)
    for jj, (train_index, val_index) in enumerate(kf.split(train_features)):
        print("Fitting fold", jj+1)
        train_x = train_features[train_index]
        val_x = train_features[val_index]
        train_target = target[train_index,ii]
        classifier = XGBClassifier(tree_method='hist', max_depth=2)
        classifier.fit(train_x, train_target)
        train_oof[val_index, ii] = classifier.predict_proba(val_x)[:,1]
    print(roc_auc_score(target[:,ii], train_oof[:,ii]))

Fitting target 1
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9539191180634152
Fitting target 2
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9791708748108716
Fitting target 3
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9631296224358576
Fitting target 4
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9639586350732409
Fitting target 5
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.963276119225601
Fitting target 6
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9557014471439371
CPU times: user 5h 24min 57s, sys: 29.9 s, total: 5h 25min 27s
Wall time: 21min 46s
